In [1]:
from arcgis.gis import *
import itertools
gis = GIS('https://www.jdart.uscourts.gov/arcgis', "ao_admin")

In [2]:
#Identifying users and siging into AGOL (Both users need to have Administrator role)
orig_userid = "ao_admin"
new_userid = "f47c0274_123c_4320_8742_fb117e700e11"
olduser = gis.users.get(orig_userid)
newuser = User(gis, new_userid)
newuser

<User username:f47c0274_123c_4320_8742_fb117e700e11>

In [3]:
gis.users.me

<User username:ao_admin>

In [4]:
#Transferring the ownership of ALL GROUPS owned by this user
print("\n***Transferring the ownership of groups...")
usergroups = olduser.groups
for group in usergroups:
    grp = gis.groups.get(group['id'])
    if (grp.owner == orig_userid):
        grp.reassign_to(new_userid)
    else:
        grp.add_users(new_userid)

print("The ownership of groups was successfully transferred!")
# usergroups


***Transferring the ownership of groups...
The ownership of groups was successfully transferred!


In [7]:
#Transferring items in folders        
print("\n***Transferring the ownership of items in selected folders...")

folders = olduser.folders
print(folders)
num_list=[]
for folder in folders:
    # print("made it here")
    gis.content.create_folder(folder['title'], new_userid)
    print(folder)
    folderitems = olduser.items(folder=folder['title'])
    num_items=int(len(folderitems))
    num_list.append(num_items)
    print("Transferring "+ str(num_items) + " item(s) from folder: "+folder['title'])
    for item in folderitems:
        if 'View Service' in item.typeKeywords:
            print(item.title + "This View will automatically be transferred")
        elif 'Microsoft' in item.type:
            print(item.title + " - Forget it")
        elif 'Code Attachment' in item.type:
            print("Forget This type")
        else:
            itemsharing=item.shared_with
            item.unshare(groups=itemsharing['groups'])
            item.reassign_to(new_userid, target_folder=folder['title'])
            item.share(everyone=itemsharing['everyone'], org=itemsharing['org'], groups=itemsharing['groups'], allow_members_to_edit=True)
            print(f"{item} was reassigned")

    def sum_list(num_list):
        sum = 0
        for x in num_list:
            sum += x
        return sum
    total_items=sum_list(num_list)

print("\nThe ownership of " +str(total_items) + " items was successfully transferred!")


***Transferring the ownership of items in selected folders...
[{'username': 'ao_admin', 'id': '346b071594054a40ae46e373ca73ff6b', 'title': 'AKX Items', 'created': 1619544350931}, {'username': 'ao_admin', 'id': '2e2d39c77da7447e90935a7fc899016c', 'title': 'ALM Items', 'created': 1628622588024}, {'username': 'ao_admin', 'id': '16756490bc2743588e96ac29aecf4502', 'title': 'ALN Items', 'created': 1628627799955}, {'username': 'ao_admin', 'id': 'a4aa98624d34468fb44fba8a5fea4f0e', 'title': 'ALS Items', 'created': 1631133863415}, {'username': 'ao_admin', 'id': '2e461f2a85a143e29e66845062c8f132', 'title': 'Analyst Applications', 'created': 1655935920869}, {'username': 'ao_admin', 'id': '7987542e23204b9397e55f75ff56e02b', 'title': 'AO JDART Items', 'created': 1619458181857}, {'username': 'ao_admin', 'id': '8f15e12a15ae4602b94d9410d5b1f58a', 'title': 'Authoritative Layers', 'created': 1619446908092}, {'username': 'ao_admin', 'id': '000fc14ca64d46e5b7dcdb277bdc572f', 'title': 'Blue Sky', 'created'

In [8]:
#Transferring items in the root folder   
print("\n***Transferring the ownership of items in the root folder...")
userrootcontent = olduser.items()
num_root_items=int(len(userrootcontent))
rootfoldername=olduser.username+"_RootFolderNew"
gis.content.create_folder(rootfoldername, new_userid)
root_groups_list=[]
# print(userrootcontent)

for item in userrootcontent:
    print(item)
    rootitemsharing=item.shared_with
    root_group=rootitemsharing['groups']
    root_groups_list.append(root_group)
    valueToBeRemoved = []
    try:
        while True:
            root_groups_list.remove(valueToBeRemoved)
    except ValueError:
        pass

    root_groups= list(itertools.chain(*root_groups_list))

    for group in root_groups:
        grop = gis.groups.get(group['id'])
        if (grop.owner == orig_userid):
            grop.reassign_to(new_userid)
        else:
            grop.add_users(new_userid)
            grop.remove_users(orig_userid)
    if 'View Service' in item.typeKeywords:
        print(item.title + "This View will automatically be transferred")
    elif 'Microsoft' in item.type:
        print(item.title + " - Forget it")
    elif 'Code Attachment' in item.type:
        print("Forget This type")
    else: 
        item.unshare(groups=rootitemsharing['groups'])
        item.reassign_to(new_userid, target_folder=rootfoldername)
        item.share(everyone=rootitemsharing['everyone'], org=rootitemsharing['org'], groups=rootitemsharing['groups'], allow_members_to_edit=True)

# print("The ownership of "+ str(num_root_items)+" items in Root Folder was transferred successfully!")

# print ("\nTotal number of items transferred: "+str(total_items+num_root_items)+" items") 

# print("\n***Process was completed!***")


***Transferring the ownership of items in the root folder...
